### RAG

In [2]:
__import__('pysqlite3')

import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [4]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core.memory import MemoryContent, MemoryMimeType
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.memory.chromadb import ChromaDBVectorMemory, PersistentChromaDBVectorMemoryConfig
import os

from dotenv import load_dotenv

load_dotenv(override=True)

True

In [5]:
sources = [
    "https://raw.githubusercontent.com/microsoft/autogen/refs/heads/main/README.md",
    "https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/teams.html",
    "https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/agents.html",
    "https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/termination.html"
]

In [6]:
chroma_user_memory = ChromaDBVectorMemory(
    config=PersistentChromaDBVectorMemoryConfig(
        collection_name="user_memory",
        persistence_path=os.getenv("CHROMA_PERSISTENCE_PATH", "user_memory"),
        k=3,
        score_threshold=0.5,
    )
)

In [7]:
await chroma_user_memory.clear()

Failed to send telemetry event ClientStartEvent: module 'chromadb' has no attribute 'get_settings'
Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'
Failed to send telemetry event CollectionGetEvent: module 'chromadb' has no attribute 'get_settings'


In [8]:
from index_utils import SimpleDocumentIndexer


async def index_autogen_docs() -> None:
    indexer = SimpleDocumentIndexer(
        memory=chroma_user_memory,
    )

    chunks: int = await indexer.index_documents(sources)

    print(f"Indexed {chunks} chunks from {len(sources)} documents.")

await index_autogen_docs()

/home/vscode/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:10<00:00, 8.17MiB/s]
Failed to send telemetry event CollectionAddEvent: module 'chromadb' has no attribute 'get_settings'


Indexed 69 chunks from 4 documents.


In [9]:
rag_assistant = AssistantAgent(
    name="RAGAssistant",
    description="An assistant that can answer questions using the RAG approach.",
    model_client=AzureOpenAIChatCompletionClient(
        azure_deployment=os.getenv("deployment_name"),
        model=os.getenv("model_name"),
        api_version=os.getenv("api_version"),
        azure_endpoint=os.getenv("azure_endpoint"),
        api_key=os.getenv("api_key")
    ),
    memory = [chroma_user_memory],
    system_message="You are a helpful assistant that can answer questions using the RAG approach. "
                   "You have access to a memory that contains relevant information.",
)

In [10]:
stream = rag_assistant.run_stream(
    task="What is Agent Chat in AutoGen?"
)

await Console(stream)

await chroma_user_memory.close()

print("RAG Assistant has finished running. You can now ask questions using the RAG approach.")

---------- TextMessage (user) ----------
What is Agent Chat in AutoGen?


Failed to send telemetry event CollectionQueryEvent: module 'chromadb' has no attribute 'get_settings'


---------- MemoryQueryEvent (RAGAssistant) ----------
[MemoryContent(content='arch Literature Review API Reference PyPi Source AgentChat Agents Agents # AutoGen AgentChat provides a set of preset Agents, each with variations in how an agent might respond to messages. All agents share the following attributes and methods: name : The unique name of the agent. description : The description of the agent in text. run : The method that runs the agent given a task as a string or a list of messages, and returns a TaskResult . Agents are expected to be stateful and this method is expected to be called with new messages, not complete history . run_stream : Same as run() but returns an iterator of messages that subclass BaseAgentEvent or BaseChatMessage followed by a TaskResult as the last item. See autogen_agentchat.messages for more information on AgentChat message types. Assistant Agent # AssistantAgent is a built-in agent that uses a language model and has the ability to use tools. Warning As

/workspaces/advanced/.venv/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py:981: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(
